In [59]:
()[1:]

()

In [67]:
f = lambda:None
f.__defaults__

In [42]:
%%writefile tests/example_string_short.py
from pipedata import AutoNode as Node
from pipedata import TrackedFile, InputTrackedFile, index_file_flush
import os
_node = Node
numberFile = InputTrackedFile('number.txt')
letterFile = InputTrackedFile('letter.txt')
import time
# _output_kw = Node.from_func
@Node.from_func({
    "OUT":TrackedFile("out5.txt"),
#     "BAM":TrackedFile( "test.fastq.bam"  )
})
def out5(  self, numberFile, letterFile ):
    '''
    some doc
    '''
    number = numberFile().open('r').read().strip()
    letter = letterFile().open('r').read().strip()
    with self.output_kw['OUT'].open("w") as f:
        f.write( 5 * (number+letter)+'\n')
    return


@Node.from_func({
    "OUT":TrackedFile("out10.txt"),
#     "BAM":TrackedFile( "test.fastq.bam"  )
})
def out10(  self, numberFile, letterFile ):
    '''
    some doc
    '''
    number = numberFile().open('r').read().strip()
    letter = letterFile().open('r').read().strip()
    with self.output_kw['OUT'].open("w") as f:
        f.write( 10 * (number+letter)+'\n')
    return

@Node.from_func({
    "OUT":TrackedFile("out15.txt"),
#     "BAM":TrackedFile( "test.fastq.bam"  )
})
def out15(  self, numberFile, letterFile ):
    '''
    some doc
    '''
    number = numberFile().open('r').read().strip()
    letter = letterFile().open('r').read().strip()
    with self.output_kw['OUT'].open("w") as f:
        f.write( 15 * (number+letter)+'\n')
    return


@Node.from_func({
    'OUT':TrackedFile('combined.txt')
})
def make_combined( self, out5, out10, out15, ):
    lines = []
    [ lines.extend(list(x()['OUT'].open('r'))) for x in [out5,
                                                         out10,out15
                                                        ]]
    with self['OUT'].open('w') as f:
        map(f.write,lines)
    return 1


if __name__ == '__main__':
    
    TrackedFile.VERBOSE=0
    print('START'+20*"-")
    os.chdir("tests/")
    res = make_combined()
    index_file_flush("./test.index")
    print(res['OUT'].open('r').read())
#     open(''))
    print('END'+20*"-")
    import dill
    dill.dumps(make_combined)
    
#     make_bam()
#     make_csv()
#     print("[middle]",middle())

Overwriting tests/example_string_short.py


In [59]:
%%writefile test.py

# import pymisca.ext as pyext
import subprocess
def t1():
    test_expect = '+ mkdir -p tests\n+ ln pipedata.py tests -f\n+ rm -f tests/test.index tests/out10.txt tests/out15.txt tests/out5.txt\n+ echo 1\n+ echo a\n+ python2 tests/example_string_short.py\nSTART--------------------\nRUNNING:<Node with func:make_combined>\nRUNNING:<Node with func:out5>\nRUNNING:<Node with func:out10>\nRUNNING:<Node with func:out15>\n1a1a1a1a1a\n1a1a1a1a1a1a1a1a1a1a\n1a1a1a1a1a1a1a1a1a1a1a1a1a1a1a\n\nEND--------------------\n+ python2 tests/example_string_short.py\nSTART--------------------\nSKIPPING:<Node with func:make_combined>\n1a1a1a1a1a\n1a1a1a1a1a1a1a1a1a1a\n1a1a1a1a1a1a1a1a1a1a1a1a1a1a1a\n\nEND--------------------\n+ echo 2\n+ echo b\n+ python2 tests/example_string_short.py\nSTART--------------------\nRUNNING:<Node with func:make_combined>\nRUNNING:<Node with func:out5>\nRUNNING:<Node with func:out10>\nRUNNING:<Node with func:out15>\n2b2b2b2b2b\n2b2b2b2b2b2b2b2b2b2b\n2b2b2b2b2b2b2b2b2b2b2b2b2b2b2b\n\nEND--------------------'


    test_cmd  = '''
    set -x
    exec 2>&1

    mkdir -p tests
    ln pipedata.py tests -f 
    rm -f tests/test.index tests/out*.txt

    echo "1"> tests/number.txt; 
    echo a>tests/letter.txt; 
    python2 tests/example_string_short.py


    python2 tests/example_string_short.py

    echo "2">tests/number.txt; 
    echo b>tests/letter.txt; 
    python2 tests/example_string_short.py
    '''
    print repr(subprocess.check_output(test_cmd,shell=1).strip())
    assert subprocess.check_output(test_cmd,shell=1).strip()==test_expect.strip()
t1()


Overwriting test.py


In [58]:
!python2 test.py

+ exec
'+ mkdir -p tests\n+ ln pipedata.py tests -f\n+ rm -f tests/test.index tests/out10.txt tests/out15.txt tests/out5.txt\n+ echo 1\n+ echo a\n+ python2 tests/example_string_short.py\nSTART--------------------\nRUNNING:<Node with func:make_combined>\nRUNNING:<Node with func:out5>\nRUNNING:<Node with func:out10>\nRUNNING:<Node with func:out15>\n1a1a1a1a1a\n1a1a1a1a1a1a1a1a1a1a\n1a1a1a1a1a1a1a1a1a1a1a1a1a1a1a\n\nEND--------------------\n+ python2 tests/example_string_short.py\nSTART--------------------\nSKIPPING:<Node with func:make_combined>\n1a1a1a1a1a\n1a1a1a1a1a1a1a1a1a1a\n1a1a1a1a1a1a1a1a1a1a1a1a1a1a1a\n\nEND--------------------\n+ echo 2\n+ echo b\n+ python2 tests/example_string_short.py\nSTART--------------------\nRUNNING:<Node with func:make_combined>\nRUNNING:<Node with func:out5>\nRUNNING:<Node with func:out10>\nRUNNING:<Node with func:out15>\n2b2b2b2b2b\n2b2b2b2b2b2b2b2b2b2b\n2b2b2b2b2b2b2b2b2b2b2b2b2b2b2b\n\nEND--------------------'
+ exec


In [31]:
%%bash 
set -x
exec 2>&1

mkdir -p tests
ln pipeline_tracker.py tests -f 
rm -f tests/test.index tests/out*.txt

echo "1"> tests/number.txt; 
echo a>tests/letter.txt; 
python2 tests/example_string_shaort.py


python2 tests/example_string_short.py

echo "2">tests/number.txt; 
echo b>tests/letter.txt; 
python2 tests/example_string_short.py



+ mkdir -p tests
+ ln pipeline_tracker.py tests -f
+ rm -f tests/test.index tests/out10.txt tests/out15.txt tests/out5.txt
+ echo 1
+ echo a
+ python2 tests/example_string_short.py
START--------------------
RUNNING:<Node with func:<function make_combined at 0x7f01957d8140>>
RUNNING:<Node with func:<function out5 at 0x7f01957d3c08>>
RUNNING:<Node with func:<function out10 at 0x7f01957d35f0>>
RUNNING:<Node with func:<function out15 at 0x7f01957d3ed8>>
1a1a1a1a1a
1a1a1a1a1a1a1a1a1a1a
1a1a1a1a1a1a1a1a1a1a1a1a1a1a1a

END--------------------
+ python2 tests/example_string_short.py
START--------------------
SKIPPING:<Node with func:<function make_combined at 0x7f3307c0d140>>
1a1a1a1a1a
1a1a1a1a1a1a1a1a1a1a
1a1a1a1a1a1a1a1a1a1a1a1a1a1a1a

END--------------------
+ echo 2
+ echo b
+ python2 tests/example_string_short.py
START--------------------
RUNNING:<Node with func:<function make_combined at 0x7f699b0e8140>>
RUNNING:<Node with func:<function out5 at 0x7f699b0e3c08>>
RUNNING:<Node with func:

+ exec


In [11]:
# print (1,end='')

SyntaxError: invalid syntax (<ipython-input-11-1bd562f80a83>, line 1)

In [131]:
%%bash 
set -x
ln pipeline_tracker.py tests -f 
rm -f tests/test.index
echo "1">tests/number.txt; 
echo a>tests/letter.txt; 
time python2 tests/example_string_long.py
cat tests/combined.txt

time python2 tests/example_string_long.py
echo "2">tests/number.txt; 
echo b>tests/letter.txt; 
time python2 tests/example_string_long.py


[CheckingChange]:out5.txt.changed=1.state=ABSENT
[CheckingChange]:out10.txt.changed=1.state=ABSENT
[CheckingChange]:out15.txt.changed=1.state=ABSENT
[CheckingChange]:number.txt.changed=1.state=ABSENT
[CheckingChange]:letter.txt.changed=1.state=ABSENT
RUNNING:<Node with func:<function out5 at 0x7ff374f4cc80>>
[CheckingChange]:number.txt.changed=1.state=ABSENT
[CheckingChange]:letter.txt.changed=1.state=ABSENT
RUNNING:<Node with func:<function out10 at 0x7ff374f4c758>>
[CheckingChange]:number.txt.changed=1.state=ABSENT
[CheckingChange]:letter.txt.changed=1.state=ABSENT
RUNNING:<Node with func:<function out15 at 0x7ff374f4cf50>>
[CheckingChange]:number.txt.changed=1.state=ABSENT
[CheckingChange]:letter.txt.changed=1.state=ABSENT
RUNNING:<Node with func:<function out5 at 0x7ff374f4cc80>>
out5.txt
['1a1a1a1a1a\n', '1a1a1a1a1a1a1a1a1a1a\n', '1a1a1a1a1a1a1a1a1a1a1a1a1a1a1a\n']
1a1a1a1a1a
1a1a1a1a1a1a1a1a1a1a
1a1a1a1a1a1a1a1a1a1a1a1a1a1a1a
out5.txt [(1574977942.311284, 1574978489.4967468), (11

+ ln pipeline_tracker.py tests -f
+ rm -f tests/test.index
+ echo 1
+ echo a
+ python2 tests/example_string_long.py

real	0m0.116s
user	0m0.096s
sys	0m0.016s
+ cat tests/combined.txt
+ python2 tests/example_string_long.py

real	0m0.072s
user	0m0.068s
sys	0m0.000s
+ echo 2
+ echo b
+ python2 tests/example_string_long.py

real	0m0.069s
user	0m0.060s
sys	0m0.004s


In [23]:
%%writefile example_bio.py
from pipeline_tracker import Node,TrackedFile, InputTrackedFile
import os
_node = Node
INPUT_FILE = InputTrackedFile('test_R1_.fastq')
INPUT_GENOME = InputTrackedFile('test.fa')

# _output_kw = Node.from_func
@Node.from_func({
    "BAM":TrackedFile( "test.fastq.bam"  )
})
def make_bam(  self, INPUT_FILE ):
    '''
    some doc
    '''
#     if INPUT_FILE().changed():
    if self.upstream_changed():
        print("RUNNING:%s"%self)
        lines = list( INPUT_FILE().open('r') )
        with self.output_kw['BAM'].open("w") as f:
            f.write(str(lines))
    else:
        print("SKIPPING:%s"%self)
        pass
    return


@Node.from_func({
})
def make_csv( self, make_bam=make_bam, ):
    make_bam()
#     print make_bam()['BAM']
    return 1


if __name__ == '__main__':
    os.chdir("tests/")
    make_bam()
    make_csv()
#     print("[middle]",middle())

Overwriting example_bio.py


In [26]:
import json
# json.load(object_pairs_hook=_dict)
# json.dump()

In [15]:
!touch tests/test.index

In [ ]:
from pymisca.shell import file__notEmpty

In [51]:
# ! rm tests/
# !rm i
# !tree


In [53]:
! rm */*.index 
! python2 example_bio.py

ABSENT
[CheckingChange]:test_R1_.fastq.changed=1
RUNNING:<Node with func:<function make_bam at 0x7f34b26bcb18>>
[CheckingChange]:test_R1_.fastq.changed=0
SKIPPING:<Node with func:<function make_bam at 0x7f34b26bcb18>>


In [33]:
import os
os.stat('example.py')

posix.stat_result(st_mode=33188, st_ino=3557096, st_dev=2053, st_nlink=1, st_uid=1000, st_gid=1000, st_size=539, st_atime=1574954182, st_mtime=1567113369, st_ctime=1567113369)

In [32]:
import example_bio
reload(example_bio)
import pipeline_tracker
reload(pipeline_tracker)
from example_bio import make_bam
# make_bam.func?
make_bam.input_kw
pipeline_tracker.TrackedFile.changed??


In [113]:
# %pdb 1
!python2 example_bio.py

RUNNING:<Node with func:<function make_bam at 0x7f281dd472a8>>
RUNNING:<Node with func:<function make_bam at 0x7f281dd472a8>>


In [118]:
import inspect
def f(a,b=2,*a1,**kw):
    print 'a',a
    print 'b',b
    def g():
        return 
    return g
(args, varargs, keywords, defaults) = inspect.getargspec(f)
a= 123
f.func_defaults = (a,) + f.func_defaults
g = f()


a 123
b 2


In [113]:


map(lambda x:x(f),
    [
#     inspect.getargs,
    inspect.getargspec,
#     inspect.getargvalues,
    ]
)
f()

a 123
b 2


In [87]:
# ! curl -L https://github.com/shouldsee/synoBio/raw/master/examples/data/test_R1_.fastq | head -8 > tests/test_R1_.fastq
! python2 example_bio.py
! echo
! cd tests/ && head *.fastq *.bam
# !cat tests/*.

123 () {}
RUNNING:<function make_bam at 0x7fe6f77e5410>

==> test_R1_.fastq <==
@NB501178:126:HLCC2BGXY:1:11101:18422:1050 1:N:0:TCTCGCGC+NCCTCTAT
CACCTNGTATCACTTCCAAGGCATATGATAGTTTGATTATCTGTGGACGATTTCTTTCTCGTGCAATGCCGTTTC
+
AAAAA#EEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEE
@NB501178:126:HLCC2BGXY:1:11101:5478:1050 1:N:0:TCTCGCGC+NCCTCTAT
TCTTGNCTTTTTTCGTTTTTTTTTACTTTATTTTGCAAATCTGAATGACAATGATGCTCAAAGTGAATGTTATTCG
+
AAAAA#EEEEEEEEEEEEEAE/EE6/EEE/AAEEEEEEEEEEEE/A//A//EA/EE/EEEEEEEEEEA/EEEEEEA

==> test.fastq.bam <==
['@NB501178:126:HLCC2BGXY:1:11101:18422:1050 1:N:0:TCTCGCGC+NCCTCTAT\n', 'CACCTNGTATCACTTCCAAGGCATATGATAGTTTGATTATCTGTGGACGATTTCTTTCTCGTGCAATGCCGTTTC\n', '+\n', 'AAAAA#EEEEEEEEEEEEEEEEEEAEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEAEEEEEE\n', '@NB501178:126:HLCC2BGXY:1:11101:5478:1050 1:N:0:TCTCGCGC+NCCTCTAT\n', 'TCTTGNCTTTTTTCGTTTTTTTTTACTTTATTTTGCAAATCTGAATGACAATGATGCTCAAAGTGAATGTTATTCG\n', '+\n', 'AAAAA#EEEEEEEEEEEEEAE/EE6/EEE/AAEEEEEEEEEEEE/A//A/